In [2]:
import numpy as np
import pandas as pd
from typing import List, Tuple, Union
import matplotlib.pyplot as plt
import seaborn as sns
from modules import utils, model, metrics


In [3]:
pd.core.common.random_state(None)
seed = 105
np.random.seed(seed)

In [4]:
# def generate_data(n_rows:int=1000):
#     # column_patterns = [np.random.uniform(1, 72, n_rows),
#     #                    np.random.uniform(18, 118, n_rows),
#     #                    np.random.uniform(100, 1000, n_rows),
                       
#     #                    ]
#     data_list = [
#         np.random.uniform(1, 72, n_rows),
#         np.random.uniform(18, 118, n_rows),
#         np.random.uniform(100, 1000, n_rows),
#         np.random.choice([0,1], size = n_rows, p = [0.25, 0.75])
#                 ]
#     data_dict = {}
#     for i, d in enumerate(data_list):
#         data_dict[f"x{i}"] = d
#     data = pd.DataFrame(data_dict)

#     # print(training_data.shape, npdata.shape)
#     # print(training_data)
    
#     return data

In [5]:
#data = generate_data(6000)

url = "https://raw.githubusercontent.com/VincentGranville/Main/main/Telecom.csv"
data = pd.read_csv(url)

features = ['tenure', 'MonthlyCharges', 'TotalCharges','Churn']
data = data[features]
for col in data.select_dtypes(include=[object]).columns:
    data[col] = utils.label_encode(data[col])
# data['Churn'] = data['Churn'].map(category_to_integer)
data['TotalCharges'].replace(' ', np.nan, inplace=True)
data.dropna(subset=['TotalCharges'], inplace=True) # remove missing data
arr1 = data['tenure'].to_numpy()
arr2 = data['TotalCharges'].to_numpy()
arr2 = arr2.astype(float)
residues = arr2 - arr1 * np.sum(arr2) / np.sum(arr1) # also try arr2/arr1
data['TotalChargeResidues'] = residues

print(data.head())
print (data.shape)
print (data.columns)


   tenure  MonthlyCharges  TotalCharges  Churn  TotalChargeResidues
0       1           29.85          5487      1          5385.560402
1      34           56.95          1024      1         -2424.946340
2       2           53.85          3860      0          3657.120804
3      45           42.30          4103      1          -461.781920
4       2           70.70          1645      0          1442.120804
(7043, 5)
Index(['tenure', 'MonthlyCharges', 'TotalCharges', 'Churn',
       'TotalChargeResidues'],
      dtype='object')


In [6]:
data.head()

,tenure,MonthlyCharges,TotalCharges,Churn,TotalChargeResidues
0,1,29.85,5487,1,5385.560402
1,34,56.95,1024,1,-2424.946340
2,2,53.85,3860,0,3657.120804
3,45,42.30,4103,1,-461.781920
4,2,70.70,1645,0,1442.120804


In [7]:
features = ['tenure', 'MonthlyCharges', 'TotalChargeResidues','Churn']
data = data[features]

# Split data into Train & Validation sets
training_data  = data.sample(frac = 0.5)
validation_data = data.drop(training_data.index)
# np_tr = np.array(training_data)
# np_val = np.array(validation_data)

print(f"Data Shape: {data.shape}\nTraining Shape: {training_data.shape}\nValidation Shape: {validation_data.shape}")

Data Shape: (7043, 4)
Training Shape: (3522, 4)
Validation Shape: (3521, 4)


In [8]:
# # Get Student's Success Data
# data = utils.get_cleaned_students_data()

# # Split data into Train & Validation sets
# training_data  = data.sample(frac = 0.5)
# validation_data = data.drop(training_data.index)

# print(f"Data Shape: {data.shape}\nTraining Shape: {training_data.shape}\nValidation Shape: {validation_data.shape}")

In [9]:
# class NoGANSynth:
#     def __init__(self, data:np.array, features:List = None, 
#                  bins:List = None
#                  ):
#         self.data = np.array(data, np.float64)
#         self.nobs = len(data)
#         if isinstance(data, pd.DataFrame):
#             self.features = data.columns
#         elif features is None:
#             raise TypeError("features should not be null if data is not a Pandas Dataframe")
#         else:
#             self.features = features
#         self.n_features = len(self.features)
#         self.eps = 0.0000000001
#         if bins is None:
#             self.bins_per_feature = [ np.random.randint(3,5) for i in range(self.n_features)]
#         else:
#             self.bins_per_feature = bins

#         # self.bin_edges = np.array([(np.histogram(npdata[:,k], 
#         #                                          bins=self.bins_per_feature[k]) \
#         #                     )[1] for k in range(self.n_features)], dtype='object')
#         self.bin_edges = [np.quantile(self.data[:,k], 
#                                       np.arange(0, 1 + self.eps, 1/self.bins_per_feature[k]), 
#                                       axis=0
#                                       ) for k in range(self.n_features)]
        
#     def create_bin_keys(self):
#         # Get bin indices for each row in the data
#         bin_indices = np.array([np.clip(np.searchsorted(self.bin_edges[col], 
#                                                         self.data[:, col], side='right')-1,
#                                         0,
#                                         len(self.bin_edges[col])-2
#                                         ) for col in range(self.data.shape[1])])
        
#         bin_indices = bin_indices.T

#         # Find the counts of all unique list entries
#         unique_entries, counts = np.unique(bin_indices, axis=0, return_counts=True)

#         # Create a dictionary having each entry as key and corresponging counts and actual lists as values
#         bin_keys = {}
#         for entry, count in zip(unique_entries, counts):
#             key_str = ', '.join(map(str, entry))
#             lower_val = [self.bin_edges[k][entry[k]] for k in range(len(entry))]
#             upper_val = [self.bin_edges[k][1 + entry[k]] for k in range(len(entry))]
#             bin_keys[key_str] = {'frequency': count, 'value': entry, 
#                                     'lower_val': lower_val, 'upper_val': upper_val
#                                     }

#         self.bin_keys = bin_keys
    
#     def random_bin_counts(self,no_of_rows:int)->np.array:
#         # generate multinomial bin counts with same expectation as real counts
#         pvals = []
#         for key in self.bin_keys:
#             #print(f"bin_count[{skey}] = {bin_count[skey]}, nobs: {nobs}, bin_count[{skey}]/nobs = {bin_count[skey]/nobs}")
#             pvals.append(self.bin_keys[key]["frequency"]/self.nobs)
#         return(np.random.multinomial(no_of_rows, pvals))

#     def generate_synthetic_data(self, no_of_rows:int)->pd.DataFrame:
#         #print("*"*40 + "Generating Synthetic Data" + "*"*40)
#         bin_count_random = self.random_bin_counts(no_of_rows)
#         data_synth = []
#         for i, key in enumerate(self.bin_keys):
#             lower_val = self.bin_keys[key]["lower_val"]
#             upper_val = self.bin_keys[key]["upper_val"]
#             count = bin_count_random[i]
#             #print(key, count)
#             for j in range(count):
#                 new_obs = np.empty(self.n_features) # synthesized obs
#                 for k in range(self.n_features):
#                     new_obs[k] = np.random.uniform(lower_val[k],
#                                                    upper_val[k])   
#                     #print("adding new_obs")
#                 data_synth.append(new_obs)               
#         data_synth = pd.DataFrame(data_synth, columns = self.features)
#         return data_synth
        
            
        
        

In [10]:
# def scott_bin_size(data):
#     n = len(data)
#     h = 3.5 * np.std(data) / (n**(1/3))
#     k = int(np.ceil((np.max(data) - np.min(data)) / h))
#     return k

# def freedman_diaconis_bin_size(data):
#     n = len(data)
#     iqr = np.percentile(data, 75) - np.percentile(data, 25)
#     h = 2 * iqr / (n**(1/3))
#     k = int(np.ceil((np.max(data) - np.min(data)) / h))
#     return k

In [11]:
# def plot_histogram(data_arr:Union[list, pd.Series], bins:int=10)->None:
#     plt.hist(data_arr, bins = bins)

In [12]:
# plot_histogram(training_data.marital_status,bins=10)

In [13]:
# training_data.plot(kind='hist', subplots= True)

In [14]:
bins = [50, 40, 40, 4]
#features = training_data.columns
n_synth_rows = len(validation_data)
nogan = model.NoGANSynth(data = training_data,
                   bins = bins
                   )
nogan.create_bin_keys()
synth_data = nogan.generate_synthetic_data(n_synth_rows)
print(synth_data)

         tenure  MonthlyCharges  TotalChargeResidues  Churn
0      0.174797       19.650978          2151.527611    1.0
1      0.978910       19.897544          2310.456110    1.0
2      0.356614       19.790749          2345.845415    1.0
3      0.133221       55.969270          2250.727668    1.0
4      0.882716       61.958606          2197.984581    1.0
...         ...             ...                  ...    ...
3516  72.000000      112.018102         -1328.662824    1.0
3517  72.000000      111.499800         -1278.197748    1.0
3518  72.000000      116.048693         -1077.100989    1.0
3519  72.000000      116.045264         -1139.726006    1.0
3520  72.000000      118.169005         -1131.458518    1.0

[3521 rows x 4 columns]


In [25]:
nogan.bin_keys

{'0, 2, 29, 3': {'frequency': 2,
  'value': array([ 0,  2, 29,  3], dtype=int64),
  'lower_val': [0.0, 19.65, 2103.206178121848, 1.0],
  'upper_val': [1.0, 19.9, 2372.145122593096, 1.0]},
 '0, 8, 29, 3': {'frequency': 1,
  'value': array([ 0,  8, 29,  3], dtype=int64),
  'lower_val': [0.0, 25.15, 2103.206178121848, 1.0],
  'upper_val': [1.0, 26.567500000000006, 2372.145122593096, 1.0]},
 '0, 15, 29, 3': {'frequency': 1,
  'value': array([ 0, 15, 29,  3], dtype=int64),
  'lower_val': [0.0, 55.45, 2103.206178121848, 1.0],
  'upper_val': [1.0, 59.55, 2372.145122593096, 1.0]},
 '0, 17, 29, 3': {'frequency': 1,
  'value': array([ 0, 17, 29,  3], dtype=int64),
  'lower_val': [0.0, 61.756250000000044, 2103.206178121848, 1.0],
  'upper_val': [1.0, 65.67250000000001, 2372.145122593096, 1.0]},
 '0, 20, 29, 3': {'frequency': 1,
  'value': array([ 0, 20, 29,  3], dtype=int64),
  'lower_val': [0.0, 70.4, 2103.206178121848, 1.0],
  'upper_val': [1.0, 73.5, 2372.145122593096, 1.0]},
 '0, 25, 29, 3': 

In [15]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

# # Create a sample pandas DataFrame (replace this with your dataset)
# data = {
#     'variable1': np.random.randn(100),
#     'variable2': np.random.randn(100),
#     'variable3': np.random.randn(100)
# }
# df = pd.DataFrame(data)

# # Function to calculate ECDF for a given dataset
# def ecdf(data):
#     n = len(data)
#     x = np.sort(data)
#     y = np.arange(1, n+1) / n
#     return x, y

# # Create a new figure
# plt.figure(figsize=(10, 6))

# # Plot ECDF for each variable
# for col in df.columns:
#     x, y = ecdf(df[col])
#     plt.plot(x, y, marker='.', linestyle='none', label=col)

# # Add labels and legend
# plt.xlabel('Value')
# plt.ylabel('ECDF')
# plt.title('Multivariate ECDF')
# plt.legend()

# # Show the plot
# plt.show()


In [16]:
# def compute_ecdf(data:pd.DataFrame, n_nodes:int = 1000, 
#                  adjusted:bool = False)->List:
#     if not isinstance(data, pd.DataFrame):
#         raise TypeError("Data should be a Pandas DataFrame!!") 
    
#     eps = 0.0000000001
#     ecdf = []
#     features = data.columns
#     n_features = len(features)
#     for point in range(n_nodes):
#         if point % 100 == 0:
#             print(f"Sampling ecdf, location = {point}, adjusted = {adjusted}")
        
#         percentiles = np.random.uniform(0, 1, n_features)
#         perc_vals = [eps + np.quantile(data.iloc[:,k],perc) for k, perc in enumerate(percentiles)]
#         #print(perc_vals)
#         query_str = " and ".join([ f"{features[k]} <= {perc_val}"  for k, perc_val in enumerate(perc_vals)])
#         filter_count = len(data.query(query_str))
#         #print(filter_count)
#         if filter_count > 0:
#             key = ', '.join(map(str, perc_vals))
#             #print(key)
#             ecdf.append((key, query_str, filter_count/ len(data)))
            
#     # Sort the list based on the items (second element of each tuple)
#     ecdf.sort(key=lambda item: item[2])
#     # # Separate keys and items into separate columns
#     # arr_location = [item[0] for item in ecdf]
#     # arr_val = [item[1] for item in ecdf]

#     return ecdf

# def ks_delta(synthetic_data, ecdf_validation):
#     ks_max = 0
#     ecdf_real = []
#     ecdf_synth = []
#     for i, e_val in enumerate(ecdf_validation):
#         query_str = e_val[1]
#         filter_count = len(synthetic_data.query(query_str))
#         synth_value = filter_count / len(synthetic_data)
#         real_value = e_val[2]
#         ks = abs(real_value - synth_value)
#         ecdf_real.append(real_value)
#         ecdf_synth.append(synth_value)
#         if ks > ks_max:
#             ks_max = ks

#     return (ks_max, ecdf_real, ecdf_synth)


In [17]:
# percentiles = np.random.uniform(0, 1, n_features)
# perc_vals = [eps + np.quantile(training_data.iloc[:,k],perc) for k, perc in enumerate(percentiles)]
# perc_vals

In [18]:
ecdf_validation1 = metrics.compute_ecdf(validation_data, 1000, False)
ecdf_validation2 = metrics.compute_ecdf(validation_data, 1000, True)

Sampling ecdf, location = 0, adjusted = False
Sampling ecdf, location = 100, adjusted = False
Sampling ecdf, location = 200, adjusted = False
Sampling ecdf, location = 300, adjusted = False
Sampling ecdf, location = 400, adjusted = False
Sampling ecdf, location = 500, adjusted = False
Sampling ecdf, location = 600, adjusted = False
Sampling ecdf, location = 700, adjusted = False
Sampling ecdf, location = 800, adjusted = False
Sampling ecdf, location = 900, adjusted = False
Sampling ecdf, location = 0, adjusted = True
Sampling ecdf, location = 100, adjusted = True
Sampling ecdf, location = 200, adjusted = True
Sampling ecdf, location = 300, adjusted = True
Sampling ecdf, location = 400, adjusted = True
Sampling ecdf, location = 500, adjusted = True
Sampling ecdf, location = 600, adjusted = True
Sampling ecdf, location = 700, adjusted = True
Sampling ecdf, location = 800, adjusted = True
Sampling ecdf, location = 900, adjusted = True


In [19]:
ks_max1, ecdf_real1, ecdf_synth1 = metrics.ks_delta(synth_data, ecdf_validation1)
ks_max2, ecdf_real2, ecdf_synth2 = metrics.ks_delta(synth_data, ecdf_validation2)
ks_max = max(ks_max1, ks_max2)
print(f"ECDF Kolmogorof-Smirnov dist. (synth. vs valid.): {ks_max:6.4f}")

base_ks_max1, ecdf_val1, ecdf_train1 = metrics.ks_delta(training_data, ecdf_validation1)
base_ks_max2, ecdf_val2, ecdf_train2 = metrics.ks_delta(training_data, ecdf_validation2)
base_ks_max = max(base_ks_max1, base_ks_max2)
print(f"Base ECDF Kolmogorof-Smirnov dist. (train. vs valid.): {base_ks_max:6.4f}")

ECDF Kolmogorof-Smirnov dist. (synth. vs valid.): 0.2195
Base ECDF Kolmogorof-Smirnov dist. (train. vs valid.): 0.0328


In [20]:
# ecdf = {
#     'key3': 30,
#     'key1': 10,
#     'key2': 20,
#     'key4': 40
# }

# # Create a list of tuples containing keys and items
# sorted_list = [(key, value) for key, value in ecdf.items()]

# # Sort the list based on the items (second element of each tuple)
# sorted_list.sort(key=lambda item: item[1])

# # Separate keys and items into separate columns
# sorted_keys = [item[0] for item in sorted_list]
# sorted_items = [item[1] for item in sorted_list]

# # Print the separated keys and items
# print("Sorted Keys:", sorted_keys)
# print("Sorted Items:", sorted_items)


In [21]:
# import numpy as np
# import pandas as pd
# from scipy.stats import ks_2samp

# # Create two sample pandas DataFrames (replace with your data)
# np.random.seed(42)
# data1 = pd.DataFrame(np.random.randn(100, 4), columns=['feature1', 'feature2', 'feature3', 'feature4'])
# data2 = pd.DataFrame(np.random.randn(100, 4), columns=['feature1', 'feature2', 'feature3', 'feature4'])

# # Define the number of permutations for resampling
# num_permutations = 1000

# # Initialize an array to store permutation KS statistics for each feature
# permuted_ks_stats = {col: [] for col in data1.columns}

# # Perform permutation testing
# for _ in range(num_permutations):
#     combined_data = pd.concat([data1, data2])
#     combined_data_shuffled = combined_data.sample(frac=1, replace=False, random_state=None)
    
#     shuffled_data1 = combined_data_shuffled.iloc[:len(data1)]
#     shuffled_data2 = combined_data_shuffled.iloc[len(data1):]
    
#     for col in data1.columns:
#         permuted_ks_statistic, _ = ks_2samp(shuffled_data1[col], shuffled_data2[col])
#         permuted_ks_stats[col].append(permuted_ks_statistic)

# # Calculate the observed KS statistics for each feature
# observed_ks_stats = {col: ks_2samp(data1[col], data2[col])[0] for col in data1.columns}

# # Calculate p-values for each feature based on the permutation results
# p_values = {col: (np.sum(np.array(permuted_ks_stats[col]) >= observed_ks_stats[col]) + 1) / (num_permutations + 1) for col in data1.columns}

# # Define the significance level
# alpha = 0.05

# # Check if any of the p-values are less than the significance level to reject the null hypothesis
# features_different = [col for col in p_values if p_values[col] < alpha]

# if features_different:
#     print(f"Null hypothesis rejected for features: {', '.join(features_different)}.")
# else:
#     print("Null hypothesis not rejected. Distributions are not significantly different for any features.")


In [22]:
# bin_indices = np.array([np.clip(np.searchsorted(nogan.bin_edges[col], 
#                                                 npdata[:, col], side='right')-1,
#                                 0,
#                                 len(nogan.bin_edges[col])-2
#                                 ) for col in range(npdata.shape[1])])

# bin_indices = bin_indices.T
# bin_indices

In [23]:
# import numpy as np

# bin_edges = np.array([[ 1.15941181, 16.80380687, 32.44820193, 48.09259699, 63.73699205],
#                       [25.01982862, 43.73478738, 62.44974615, 81.16470491, 99.87966368]])
# npdata = np.array([[6.9455346, 75.70576002],
#                    [24.67894677, 42.88777188],
#                    [62.0142579, 52.15530542],
#                    [8.50376531, 25.01982862],
#                    [39.03044313, 71.8475252],
#                    [5.52195945, 70.54837281],
#                    [22.07525509, 99.87966368],
#                    [1.15941181, 86.63305836],
#                    [33.02922957, 81.0967122],
#                    [63.73699205, 55.03891334]])

# bin_indices = np.array([np.searchsorted(bin_edges[col], npdata[:, col], side='right') for col in range(npdata.shape[1])])
# print("Bin Indices:\n", bin_indices.T)

# bin_indices = bin_indices.T
# bin_indices


In [24]:
# unique_entries, counts = np.unique(bin_indices, axis=0, return_counts=True)

# print(unique_entries, counts)

# result_dict = {}
# for entry, count in zip(unique_entries, counts):
#     key_str = ', '.join(map(str, entry))
#     result_dict[key_str] = {'frequency': count, 'value': entry}
    
# print(result_dict)
